# METER DATA LOGIC

# Import and Load Data

In [19]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import meterdatalogic as ml

### From Dataframe

In [2]:
TZ = "Australia/Brisbane"

# 7 days of half-hourly data for one NMI
rng = pd.date_range("2025-01-01", periods=48*7, freq="30min", tz=TZ)
raw = pd.DataFrame({
    "t_start": rng,
    "nmi": "Q1234567890",
    "channel": "E1",
    "kwh": 0.5
})
raw.head()

df = ml.ingest.from_dataframe(raw, tz=TZ)
ml.validate.assert_canon(df)
df

,nmi,channel,flow,kwh,cadence_min
t_start,,,,,
2025-01-01 00:00:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-01 00:30:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-01 01:00:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-01 01:30:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-01 02:00:00+10:00,Q1234567890,E1,grid_import,0.5,30
...,...,...,...,...,...
2025-01-07 21:30:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-07 22:00:00+10:00,Q1234567890,E1,grid_import,0.5,30
2025-01-07 22:30:00+10:00,Q1234567890,E1,grid_import,0.5,30


### From NEM12 File

In [3]:
### This is a manual overview of NEM12 ingestion. The actual test is in test_ingest.py
# from nemreader import NEMFile
# nem_file = NEMFile("../data/Example_NEM12_ManyNMIs.zip")
# df = nem_file.get_data_frame()
# df

In [20]:
# df = ml.ingest.from_nem12("../data/unzipped/Example_NEM12_month_solar.csv", tz="Australia/Brisbane")
# df = ml.ingest.from_nem12("../data/Sample1.csv", tz="Australia/Brisbane")
df = ml.ingest.from_nem12("../data/Sample2.csv", tz="Australia/Brisbane")
# df = ml.ingest.from_nem12("../data/Example_NEM12_ManyNMIs.zip", tz="Australia/Brisbane", nmi="nmi90")
ml.validate.assert_canon(df)
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]
df

,nmi,channel,flow,kwh,cadence_min
t_start,,,,,
2015-08-03 00:00:00+10:00,SAMPLE2,E1,grid_import,0.000,30
2015-08-03 00:00:00+10:00,SAMPLE2,E2,controlled_load_import,0.000,30
2015-08-03 00:30:00+10:00,SAMPLE2,E1,grid_import,0.000,30
2015-08-03 00:30:00+10:00,SAMPLE2,E2,controlled_load_import,0.000,30
2015-08-03 01:00:00+10:00,SAMPLE2,E1,grid_import,0.000,30
...,...,...,...,...,...
2016-05-17 22:30:00+10:00,SAMPLE2,E1,grid_import,0.087,30
2016-05-17 23:00:00+10:00,SAMPLE2,E2,controlled_load_import,0.000,30
2016-05-17 23:00:00+10:00,SAMPLE2,E1,grid_import,0.113,30


In [21]:
ml.summary.summarise(df)

{'meta': {'nmis': 1,
  'start': '2015-08-03T00:00:00+10:00',
  'end': '2016-05-17T23:30:00+10:00',
  'cadence_min': 30,
  'days': 289,
  'channels': ['E1', 'E2'],
  'flows': ['controlled_load_import', 'grid_import']},
 'energy': {'controlled_load_import': 0.0, 'grid_import': 1895.43},
 'per_day_avg_kwh': 6.558581314878893,
 'peaks': {'max_interval_kwh': 1.587,
  'max_interval_time': '2016-03-08T16:30:00+10:00'},
 'profile24': [{'slot': '00:00',
   'controlled_load_import': 0.0,
   'grid_import': 0.0740242214532872},
  {'slot': '00:30',
   'controlled_load_import': 0.0,
   'grid_import': 0.07532525951557094},
  {'slot': '01:00',
   'controlled_load_import': 0.0,
   'grid_import': 0.07654325259515572},
  {'slot': '01:30',
   'controlled_load_import': 0.0,
   'grid_import': 0.07755709342560554},
  {'slot': '02:00',
   'controlled_load_import': 0.0,
   'grid_import': 0.07539446366782007},
  {'slot': '02:30',
   'controlled_load_import': 0.0,
   'grid_import': 0.07998269896193773},
  {'slot

# Visualisations

In [22]:
hist = (
    df.reset_index()[["t_start","flow","kwh"]]
      .sort_values("t_start")
)

fig = px.area(
    hist, x="t_start", y="kwh", color="flow",
    title="Historical Interval Energy (kWh) by Flow",
    labels={"t_start":"Time", "kwh":"kWh"}
)
fig.update_layout(legend_title_text="Flow", hovermode="x unified")
fig.show()


In [23]:
daily = ml.transform.groupby_day(df).reset_index().melt(id_vars="day", var_name="flow", value_name="kwh")

fig = px.bar(
    daily, x="day", y="kwh", color="flow",
    title="Daily Energy by Flow",
    labels={"day":"Day","kwh":"kWh"}
)
fig.update_layout(hovermode="x unified")
fig.show()


In [24]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
melted = monthly.melt(id_vars="month", var_name="flow", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="flow",
    title="Monthly Energy by Flow",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [25]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
melted = monthly.melt(id_vars="month", var_name="flow", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="flow",
    title="Monthly Energy by Flow",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [26]:
prof = ml.transform.profile24(df)  # columns: slot + flows
melted = prof.melt(id_vars="slot", var_name="flow", value_name="kwh")

fig = px.line(
    melted, x="slot", y="kwh", color="flow",
    title="Average 24-Hour Profile (kWh per interval)",
    labels={"slot":"Time of Day","kwh":"kWh per interval"}
)
fig.update_xaxes(type="category", tickangle=-45)
fig.update_layout(hovermode="x unified")
fig.show()


In [27]:
tou = ml.transform.tou_bins(df[df["flow"]=="grid_import"], bands)  # month + off/peak/shoulder
melted = tou.melt(id_vars="month", var_name="band", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="band",
    title="Monthly Import by ToU Band",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [28]:
demand = ml.transform.demand_window(df, start="16:00", end="21:00", days="MF")  # month, demand_kw

fig = px.bar(demand, x="month", y="demand_kw",
             title="Monthly Peak Demand (kW) in Window (MF 16:00–21:00)",
             labels={"month":"Month","demand_kw":"kW"})
fig.update_layout(hovermode="x unified")
fig.show()


In [29]:
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off", "00:00", "16:00", 22.0),
        ml.types.ToUBand("peak","16:00", "21:00", 45.0),
        ml.types.ToUBand("shoulder","21:00", "24:00", 28.0),
    ],
    demand=ml.types.DemandCharge("16:00","21:00","MF", 12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

cost = ml.pricing.estimate_monthly_cost(df, plan)  # month, components, total
melted = cost.melt(id_vars="month", value_vars=["energy_cost","demand_cost","fixed_cost","feed_in_credit"],
                   var_name="component", value_name="AUD")

fig = px.bar(
    melted, x="month", y="AUD", color="component",
    title="Monthly Cost Breakdown",
    labels={"month":"Month","AUD":"$"},
    barmode="relative"  # feed_in_credit is negative → subtracts
)
# Optional: add total as line
fig2 = go.Figure(fig.data)
fig2.add_trace(go.Scatter(x=cost["month"], y=cost["total"], name="Total", mode="lines+markers"))
fig2.update_layout(title="Monthly Cost Breakdown (+ Total)", hovermode="x unified")
fig2.show()


In [14]:
g = df.copy()
g["weekday"] = g.index.weekday  # 0=Mon
g["hour"] = g.index.hour
heat = g.groupby(["weekday","hour"])["kwh"].mean().reset_index()

fig = px.density_heatmap(
    heat, x="hour", y="weekday", z="kwh",
    title="Average kWh by Hour × Weekday",
    labels={"hour":"Hour of Day","weekday":"Day (0=Mon)","kwh":"kWh per interval"},
    nbinsx=24, nbinsy=7, histfunc="avg", color_continuous_scale="Viridis"
)
fig.update_yaxes(dtick=1)
fig.show()


# Scenario Modelling

In [15]:
import pandas as pd
import meterdatalogic as ml
from meterdatalogic.scenario import run
from meterdatalogic.types import EVConfig, PVConfig, BatteryConfig

# Plan for costs (optional)
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off","00:00","16:00",22.0),
        ml.types.ToUBand("peak","16:00","21:00",45.0),
        ml.types.ToUBand("shoulder","21:00","24:00",28.0),
    ],
    demand=ml.types.DemandCharge("16:00","21:00","MF",12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

# Scenario configs
ev_cfg = EVConfig(daily_kwh=8.0, max_kw=7.0, window_start="18:00", window_end="22:00", days="ALL", strategy="immediate")
pv_cfg = PVConfig(system_kwp=6.6, inverter_kw=5.0, loss_fraction=0.15)
bat_cfg = BatteryConfig(capacity_kwh=10.0, max_kw=5.0, round_trip_eff=0.9, soc_min=0.1, soc_max=0.95)

# Run scenario
result = run(df, ev=ev_cfg, pv=pv_cfg, battery=bat_cfg, plan=plan)

print("Δ Import (kWh):", round(result.delta["import_kwh_delta"], 2))
print("Δ Export (kWh):", round(result.delta["export_kwh_delta"], 2))
print("Δ Cost ($):", None if result.delta["cost_total_delta"] is None else round(result.delta["cost_total_delta"], 2))
print("Explain:", {k: round(v,2) if isinstance(v,(int,float)) and v is not None else v for k,v in result.explain.items()})

Δ Import (kWh): 53.14
Δ Export (kWh): 20761.92
Δ Cost ($): -1030.87
Explain: {'ev_kwh': 2312.0, 'pv_kwh': 22583.9, 'battery_discharge_kwh': 2332.32, 'battery_charge_kwh': 2591.47, 'battery_cycles_est': 233.23, 'pv_self_consumption_pct': 8.07}


In [16]:
# Optional: charts (Plotly)
import plotly.express as px

before_m = ml.transform.groupby_month(result.df_before).melt(id_vars="month", var_name="flow", value_name="kwh")
after_m = ml.transform.groupby_month(result.df_after).melt(id_vars="month", var_name="flow", value_name="kwh")
before_m["scenario"] = "before"; after_m["scenario"] = "after"
both = pd.concat([before_m, after_m], ignore_index=True)

fig = px.bar(both, x="month", y="kwh", color="flow", facet_row="scenario",
             barmode="stack", title="Monthly Energy by Flow — Before vs After")
fig.show()


In [17]:
# Average 24-hour profile: baseline vs scenario (overlayed)
# — shows import/export as separate traces, plus an optional "net" line.

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

try:
    import meterlogic as ml
except ImportError:
    import meterdatalogic as ml

def plot_avg_profile_before_after(df_before: pd.DataFrame,
                                  df_after: pd.DataFrame,
                                  *,
                                  show_net: bool = True,
                                  title: str = "Average 24-Hour Profile — Before vs After"):
    """
    Overlays baseline and scenario average day profiles.
    - Uses ml.transform.profile24() to compute kWh per interval slot (mean across days)
    - Plots grid_import and grid_export_solar as separate traces
    - Optionally adds a 'net' line (import - export) for each scenario
    """
    # Compute profile (slot + columns per flow)
    prof_b = ml.transform.profile24(df_before)  # columns: slot, grid_import, grid_export_solar, ...
    prof_a = ml.transform.profile24(df_after)

    # Ensure both have the same slot ordering (categorical 00:00..23:30)
    slots = prof_b["slot"].unique().tolist()
    prof_a = prof_a.set_index("slot").reindex(slots).reset_index()

    # Helper to build a tidy frame with optional net
    def melt_with_net(prof: pd.DataFrame, label: str) -> pd.DataFrame:
        flows = [c for c in prof.columns if c != "slot"]
        tidy = prof.melt(id_vars="slot", value_vars=flows, var_name="flow", value_name="kwh")
        tidy["scenario"] = label

        if show_net:
            imp = prof.get("grid_import", pd.Series(0.0, index=prof.index))
            exp = prof.get("grid_export_solar", pd.Series(0.0, index=prof.index))
            net = imp - exp  # positive = net import, negative = net export
            tidy = pd.concat([
                tidy,
                pd.DataFrame({"slot": prof["slot"], "flow": "net", "kwh": net, "scenario": label})
            ], ignore_index=True)

        return tidy

    tb = melt_with_net(prof_b, "before")
    ta = melt_with_net(prof_a, "after")
    both = pd.concat([tb, ta], ignore_index=True)

    # Optional: flip export sign so export appears below zero (purely visual)
    both["kwh_plot"] = np.where(both["flow"].str.contains("export"), -both["kwh"], both["kwh"])

    # Build the figure: overlay 'before' and 'after'
    fig = go.Figure()

    def add_lines(df_scn, name_suffix, dash=None, width=2):
        for flow_name in sorted(df_scn["flow"].unique()):
            sub = df_scn[df_scn["flow"] == flow_name]
            fig.add_trace(go.Scatter(
                x=sub["slot"], y=sub["kwh_plot"],
                mode="lines",
                name=f"{flow_name} ({name_suffix})",
                line=dict(dash=dash, width=width)
            ))

    add_lines(both[both["scenario"] == "before"], "before", dash="dot", width=2)
    add_lines(both[both["scenario"] == "after"],  "after",  dash=None,   width=3)

    fig.update_layout(
        title=title,
        xaxis_title="Time of Day",
        yaxis_title="kWh per interval",
        hovermode="x unified",
        legend_title_text="Series",
    )
    fig.update_xaxes(type="category", tickangle=-45)
    fig.add_hline(y=0, line_color="black", line_width=0.6)

    return fig

# --- Usage example ---
# Assuming you already have `result = run(df, ev=..., pv=..., battery=..., plan=...)`:
fig = plot_avg_profile_before_after(result.df_before, result.df_after, show_net=True)
fig.show()


In [18]:
import plotly.graph_objects as go
import pandas as pd

# Convert profile24 lists to DataFrames if they come as lists of dicts
df_before = pd.DataFrame(result.summary_before["profile24"])
df_after = pd.DataFrame(result.summary_after["profile24"])

# Ensure slot (HH:MM) column is sorted in time order
df_before = df_before.sort_values("slot")
df_after = df_after.sort_values("slot")

# Create Plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_before["slot"],
    y=df_before["grid_import"],
    mode="lines+markers",
    name="Before Scenario",
    line=dict(color="royalblue"),
    marker=dict(size=4)
))

fig.add_trace(go.Scatter(
    x=df_after["slot"],
    y=df_after["grid_import"],
    mode="lines+markers",
    name="After Scenario",
    line=dict(color="orange"),
    marker=dict(size=4)
))

fig.update_layout(
    title="Average Daily Profile – Before vs After Scenario",
    xaxis_title="Time of Day",
    yaxis_title="Grid Import (kWh per 30-min interval)",
    template="plotly_white",
    hovermode="x unified",
    legend=dict(x=0.02, y=0.98)
)

fig.show()


In [25]:
import pandas as pd
import plotly.graph_objects as go
from meterdatalogic.scenario import apply_ev, apply_pv, apply_battery_self_consume, EVConfig, PVConfig, BatteryConfig

def build_solar_soak_timeseries(df, ev_cfg, pv_cfg, bat_cfg):
    """
    Recompute scenario components to visualise PV allocation (per-interval).
    Returns a tidy DataFrame with:
      - pv_gen_kwh, pv_to_load_kwh, pv_to_battery_kwh, pv_export_kwh
      - load_total_kwh (baseline + EV), grid_import_after_kwh
      - battery_discharge_kwh, battery_soc_kwh
    """
    # --- Baseline import/export series (per your canon) ---
    def _collapse_flows(df):
        idx_full = df.index
        flows = df["flow"].astype(str)
        df_imp = df.loc[flows.str.contains("import", na=False)]
        df_exp = df.loc[flows.str.contains("export", na=False)]
        imp = df_imp.groupby(level=0)["kwh"].sum() if not df_imp.empty else pd.Series(dtype=float)
        exp = df_exp.groupby(level=0)["kwh"].sum() if not df_exp.empty else pd.Series(dtype=float)
        imp = imp.reindex(idx_full, fill_value=0.0).sort_index()
        exp = exp.reindex(idx_full, fill_value=0.0).sort_index()
        return imp, exp

    # Interval hours from canon
    cmin = int(df["cadence_min"].iloc[0]) if len(df) else 30
    interval_h = cmin / 60.0

    s_import0, s_export0 = _collapse_flows(df)
    idx = s_import0.index

    # EV load (kWh/interval)
    ev_series = apply_ev(idx, ev_cfg, interval_h) if ev_cfg else pd.Series(0.0, index=idx)
    load_total = (s_import0 + ev_series).to_numpy()  # local load before PV/battery

    # PV generation (kWh/interval)
    pv_series = apply_pv(idx, pv_cfg, interval_h) if pv_cfg else pd.Series(0.0, index=idx)
    pv_arr = pv_series.to_numpy()

    # PV allocation before battery: self-consume vs leftover
    pv_to_load = np.minimum(pv_arr, load_total)
    pv_leftover = pv_arr - pv_to_load  # candidate for battery charge or export

    # Import before battery (still needed after PV-to-load)
    import_prebat = load_total - pv_to_load
    export_prebat = s_export0.to_numpy() + pv_leftover.copy()

    # Battery (self consume)
    bat_dis = bat_ch = soc = np.zeros(len(idx))
    if bat_cfg and bat_cfg.capacity_kwh > 0 and bat_cfg.max_kw > 0:
        bat_dis, bat_ch, soc = apply_battery_self_consume(
            import_prebat=import_prebat,
            pv_excess_prebat=pv_leftover,
            cfg=bat_cfg,
            interval_h=interval_h,
        )
        # export_prebat already reduced inside dispatcher via pv_leftover mutation

    # After-battery import/export
    grid_import_after = pd.Series(import_prebat, index=idx)
    pv_export_after = pd.Series(export_prebat, index=idx)

    # PV-to-battery is exactly battery charge energy (from PV excess by design)
    pv_to_battery = pd.Series(bat_ch, index=idx)

    out = pd.DataFrame({
        "pv_gen_kwh": pv_series,
        "pv_to_load_kwh": pd.Series(pv_to_load, index=idx),
        "pv_to_battery_kwh": pv_to_battery,
        "pv_export_kwh": pv_export_after,
        "load_total_kwh": pd.Series(load_total, index=idx),
        "grid_import_after_kwh": grid_import_after,
        "battery_discharge_kwh": pd.Series(bat_dis, index=idx),
        "battery_soc_kwh": pd.Series(soc, index=idx),
    })
    return out

def solar_soak_profile_plot(df, ev_cfg, pv_cfg, bat_cfg, title="Solar Soak – Average Day"):
    """
    Builds an average-day (HH:MM) plot showing PV allocation: PV->Load, PV->Battery, PV->Export (stacked area),
    with lines for Load Total and Grid Import (after battery).
    """
    ts = build_solar_soak_timeseries(df, ev_cfg, pv_cfg, bat_cfg)
    # Average by slot (time-of-day). We’ll build a "slot" column first.
    local = ts.index.tz_convert(ts.index.tz)
    slot = local.strftime("%H:%M")
    prof = ts.copy()
    prof["slot"] = slot
    g = prof.groupby("slot", sort=True).mean(numeric_only=True).reset_index()

    # Order slots chronologically (string sorting works for HH:MM)
    g = g.sort_values("slot")

    fig = go.Figure()

    # Stacked areas for PV allocation
    fig.add_trace(go.Scatter(
        x=g["slot"], y=g["pv_to_load_kwh"],
        name="PV → Load",
        mode="lines",
        stackgroup="pv", groupnorm="",
        hovertemplate="PV → Load: %{y:.3f} kWh<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=g["slot"], y=g["pv_to_battery_kwh"],
        name="PV → Battery",
        mode="lines",
        stackgroup="pv",
        hovertemplate="PV → Battery: %{y:.3f} kWh<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=g["slot"], y=g["pv_export_kwh"],
        name="PV → Export",
        mode="lines",
        stackgroup="pv",
        hovertemplate="PV → Export: %{y:.3f} kWh<extra></extra>"
    ))

    # Overlay lines for load & grid import after battery
    fig.add_trace(go.Scatter(
        x=g["slot"], y=g["load_total_kwh"],
        name="Load (Baseline + EV)",
        mode="lines+markers",
        hovertemplate="Load: %{y:.3f} kWh<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=g["slot"], y=g["grid_import_after_kwh"],
        name="Grid Import (After Battery)",
        mode="lines+markers",
        hovertemplate="Grid Import: %{y:.3f} kWh<extra></extra>"
    ))

    fig.update_layout(
        title=title,
        xaxis_title="Time of Day",
        yaxis_title="Energy per interval (kWh)",
        template="plotly_white",
        hovermode="x unified",
        legend=dict(x=0.02, y=0.98)
    )
    return fig

# Example usage:
fig = solar_soak_profile_plot(df, ev_cfg, pv_cfg, bat_cfg)
fig.show()
